<a href="https://colab.research.google.com/github/micaelCZ/Paper_Repositorio/blob/main/EscenarioB_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install tensorflow-addons
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall
from keras.callbacks import EarlyStopping



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
# Load the data
url = 'https://raw.githubusercontent.com/micaelCZ/Paper_Repositorio/main/dataset/datasetPreprocesado/Escenario2.csv'
df = pd.read_csv(url)
df = pd.read_csv(url,low_memory=False)
dataframe = pd.read_csv(url,low_memory=False)

In [59]:
# Normalise the data
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df

In [60]:
# Split the data into train and test sets
y = df['label']
label_map = {'AUDIO': 0, 'BROWSING': 1, 'CHAT': 2, 'FILE-TRANSFER' : 3, 'MAIL' : 4, 'P2P' : 5, 'VIDEO' : 6, 'VOIP' : 7}
change_labels = lambda x: label_map[x] if x in label_map else -1
y = df['label'].apply(change_labels)

In [61]:
X = df.iloc[:, 4:-1]
X = dfNormalize(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [62]:
# Reshape the data for Conv1D
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))


In [63]:
# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [64]:
# Define the model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    keras.layers.MaxPool1D(pool_size=2),
    keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    keras.layers.MaxPool1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8, activation='softmax')
])

In [65]:

from tensorflow_addons.metrics import F1Score


In [66]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=8, average='macro')])


In [67]:
# Set up EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5)


In [68]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop])


Epoch 1/10
202/202 [==============================] - 5s 15ms/step - loss: 1.6079 - accuracy: 0.4200 - precision_3: 0.5472 - recall_3: 0.1305 - f1_score: 0.1828 - val_loss: 1.3815 - val_accuracy: 0.4711 - val_precision_3: 0.6419 - val_recall_3: 0.2896 - val_f1_score: 0.1905
Epoch 2/10
202/202 [==============================] - 4s 21ms/step - loss: 1.4108 - accuracy: 0.4740 - precision_3: 0.6318 - recall_3: 0.2362 - f1_score: 0.2450 - val_loss: 1.3047 - val_accuracy: 0.5078 - val_precision_3: 0.6248 - val_recall_3: 0.3188 - val_f1_score: 0.2630
Epoch 3/10
202/202 [==============================] - 4s 19ms/step - loss: 1.2923 - accuracy: 0.5197 - precision_3: 0.7099 - recall_3: 0.3064 - f1_score: 0.3086 - val_loss: 1.1582 - val_accuracy: 0.5625 - val_precision_3: 0.7069 - val_recall_3: 0.3673 - val_f1_score: 0.3570
Epoch 4/10
202/202 [==============================] - 4s 20ms/step - loss: 1.1800 - accuracy: 0.6025 - precision_3: 0.7687 - recall_3: 0.3683 - f1_score: 0.4091 - val_loss: 1.

In [69]:
# Evaluate the model on the test set
test_loss, test_accuracy, test_precision, test_recall, test_f1 = model.evaluate(X_test, y_test)

# Multiply the metrics by 100 to show them on a 0-100 scale
test_accuracy *= 100
test_precision *= 100
test_recall *= 100
test_f1 *= 100


51/51 [==============================] - 0s 3ms/step - loss: 0.8791 - accuracy: 0.6979 - precision_3: 0.8256 - recall_3: 0.5444 - f1_score: 0.4977


In [70]:
# Print the test set metrics
print('Test set metrics:')
print('  Loss:     {:.3f}'.format(test_loss))
print('  Accuracy: {:.3f}'.format(test_accuracy))
print('  Precision:{:.3f}'.format(test_precision))
print('  Recall:   {:.3f}'.format(test_recall))
print('  F1 Score: {:.3f}'.format(test_f1))


Test set metrics:
  Loss:     0.879
  Accuracy: 69.795
  Precision:82.564
  Recall:   54.444
  F1 Score: 49.774
